In [3]:
import pybel
import pandas as pd
from pybel.dsl import Protein
from pybel.dsl import Abundance
from pybel.dsl import Pathology
from pybel.dsl import BiologicalProcess
from pybel.dsl import Population
from pybel.dsl import Gene
from pybel.dsl import Rna
from pybel.dsl import MicroRna
from pybel.dsl import Fragment
import chembl_webresource_client
import openpyxl
import networkx as nx
from pybel.io.jupyter import to_jupyter
import matplotlib.pyplot as plt
import chembl_webresource_client
from chembl_webresource_client.new_client import new_client
import pubchempy
import pickle
import re

In [ ]:
target = new_client.target
m1 = target.filter(chembl_id='CHEMBL1221984').only('target_components')
list(m1)

In [4]:
infile = open('chem2Gene','rb')
chem2Gene = pickle.load(infile)
infile.close()

In [5]:
infile = open('data/normalized_data/itmp_chem_act_3','rb')
ActList = pickle.load(infile)
infile.close()

In [6]:
infile = open('data/normalized_data/itmp_chem_moa','rb')
mechList = pickle.load(infile)
infile.close()

In [7]:
infile = open('data/normalized_data/Chem2Gene2Path','rb')
chem2Gene_all = pickle.load(infile)
infile.close()

In [ ]:
mechList

In [ ]:
protein_MechList = []
for item in mechList:
    for j in range(len(mechList[item])):
        protein_MechList.append(mechList[item][j]['target_chembl_id'])


In [ ]:
protein_MechList = set(protein_MechList)

In [ ]:
protein_MechList = filter(None, protein_MechList)

In [ ]:
None in protein_MechList

In [ ]:
protein_ActList = []
for item in ActList:
    for j in range(len(ActList[item])):
        protein_ActList.append(ActList[item][j]['target_chembl_id'])

    

In [ ]:
len(protein_ActList)

In [ ]:
protein_ActList = set(protein_ActList)

In [ ]:
protein_ActList = filter(None,protein_ActList)

In [7]:
def Ret_chembl_protein(sourceList):
    protein_List = []
    for item in sourceList:
        for j in range(len(sourceList[item])):
            protein_List.append(sourceList[item][j]['target_chembl_id'])
            
    protein_List = set(protein_List)
    protein_List = list(filter(None,protein_List))
    return(protein_List)

In [8]:
protein_mechList = Ret_chembl_protein(mechList)
len(protein_mechList)

647

In [9]:
protein_ActList = Ret_chembl_protein(ActList)
len(protein_ActList)

838

In [10]:
protein_All = protein_ActList + protein_mechList
len(protein_All)

1485

In [11]:
len(protein_All)

1485

In [12]:
protein_All[407]

'CHEMBL4641'

In [ ]:
getTarget = new_client.target
xx = getTarget.filter(chembl_id=protein_All[138]).only('target_components')
xx[0]['target_components'][0]['target_component_synonyms']

In [ ]:
'accession' in xx[0]['target_components'][0]

In [ ]:
xx[0]['target_components'][0]['accession']

In [ ]:
protein_All[966]

In [ ]:
getTarget = new_client.target
chem = getTarget.filter(chembl_id='CHEMBL2364096').only('target_components')
chem[0]['target_components'][0]['accession'] == None


In [54]:
def chembl2uniprot(chemblIDs,count):
    getTarget = new_client.target
    chem2Gene2path = []
    chemHasNoPath = []
    chemNotprotein = []
    for i in range(len(chemblIDs)):
        #print(count)
        count=count+1
        chem2path = []
        chem = getTarget.filter(chembl_id=chemblIDs[i]).only('target_components')
        #print(chem)
        #break            
        try:
            uprot_id = chem[0]['target_components'][0]['accession']
        except IndexError:
            #print(chemblIDs[i])
            chemHasNoPath.append(chemblIDs[i])
            continue
            
        if chem[0]['target_components'][0]['accession'] == None:
            chemHasNoPath.append(chemblIDs[i])
            #continue
        
    chemblIDs_clean = [item for item in chemblIDs if item not in chemHasNoPath]
    print('old',len(chemblIDs_clean))
    for i in range(len(chemblIDs_clean)):

        chem = getTarget.filter(chembl_id=chemblIDs_clean[i]).only('target_components')
        #print(chem)
        #break
        getGene = chem[0]['target_components'][0]['target_component_synonyms']
        #rint(getGene)
        #break
        try:
            getGene = [item for item in getGene if item["syn_type"] == "GENE_SYMBOL"][0]
        except IndexError:
            chemNotprotein.append(chemblIDs_clean[i])
            continue
    chemblIDs_clean = [item for item in chemblIDs_clean if item not in chemNotprotein]
    print('newLen',len(chemblIDs_clean))
    print(len(chemNotprotein))
    #break
        #print(getGene)
        #break
    for i in range(len(chemblIDs_clean)):
        
        chem = getTarget.filter(chembl_id=chemblIDs_clean[i]).only('target_components')
        #print(chem)
        #break
        getGene = chem[0]['target_components'][0]['target_component_synonyms']
        getGene = [item for item in getGene if item["syn_type"] == "GENE_SYMBOL"][0]
       
        chem2path = [item for item in chem[0]['target_components'][0]['target_component_xrefs'] if item["xref_src_db"] == "Reactome"]
        
        chem2path.append(getGene)
        print(chem2path)
        #break
        chem2Gene2path.append(chem2path)
        #print(chem2Gene2path)
        #break
    
        #print(chem2Gene2path)
        
    #print(chemHasNoPath)
    named_chem2Gene2path = dict(zip(chemblIDs_clean,chem2Gene2path))
    named_chem2Gene2path = {k: v for k, v in named_chem2Gene2path.items() if v}
    return(named_chem2Gene2path)
            

In [55]:
#chem2Gene = chembl2uniprot(proteinList,0)
#chem2Gene_all = chembl2uniprot(protein_All,0)
#a=protein_All[407:410]
#print(a)
chem2Gene_all = chembl2uniprot(protein_All,0)

old 1451
newLen 1434
17
[{'xref_id': 'R-HSA-110362', 'xref_name': 'POLB-Dependent Long Patch Base Excision Repair.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-162594', 'xref_name': 'Early Phase of HIV Life Cycle.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-174414', 'xref_name': 'Processive synthesis on the C-strand of the telomere.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-5358565', 'xref_name': 'Mismatch repair (MMR) directed by MSH2:MSH6 (MutSalpha).', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-5358606', 'xref_name': 'Mismatch repair (MMR) directed by MSH2:MSH3 (MutSbeta).', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-5651801', 'xref_name': 'PCNA-Dependent Long Patch Base Excision Repair.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-5696397', 'xref_name': 'Gap-filling DNA repair synthesis and ligation in GG-NER.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-6782210', 'xref_name': 'Gap-filling DNA repair synthesis and ligation in TC-NER.', 'xref_src_db':

[{'xref_id': 'R-HSA-3295583', 'xref_name': 'TRP channels.', 'xref_src_db': 'Reactome'}, {'component_synonym': 'TRPV1', 'syn_type': 'GENE_SYMBOL'}]
[{'xref_id': 'R-HSA-202733', 'xref_name': 'Cell surface interactions at the vascular wall.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-6798695', 'xref_name': 'Neutrophil degranulation.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-8950505', 'xref_name': 'Gene and protein expression by JAK-STAT signaling after Interleukin-12 stimulation.', 'xref_src_db': 'Reactome'}, {'component_synonym': 'MIF', 'syn_type': 'GENE_SYMBOL'}]
[{'xref_id': 'R-HSA-198753', 'xref_name': 'ERK/MAPK targets.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-199920', 'xref_name': 'CREB phosphorylation.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-2559582', 'xref_name': 'Senescence-Associated Secretory Phenotype (SASP).', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-437239', 'xref_name': 'Recycling pathway of L1.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R

[{'xref_id': 'R-HSA-417973', 'xref_name': 'Adenosine P1 receptors.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-418594', 'xref_name': 'G alpha (i) signalling events.', 'xref_src_db': 'Reactome'}, {'component_synonym': 'ADORA3', 'syn_type': 'GENE_SYMBOL'}]
[{'xref_id': 'R-HSA-1442490', 'xref_name': 'Collagen degradation.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-1474228', 'xref_name': 'Degradation of the extracellular matrix.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-1592389', 'xref_name': 'Activation of Matrix Metalloproteinases.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-6798695', 'xref_name': 'Neutrophil degranulation.', 'xref_src_db': 'Reactome'}, {'component_synonym': 'MMP8', 'syn_type': 'GENE_SYMBOL'}]
[{'component_synonym': 'CLK1', 'syn_type': 'GENE_SYMBOL'}]
[{'xref_id': 'R-HSA-1368082', 'xref_name': 'RORA activates gene expression.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-1368108', 'xref_name': 'BMAL1:CLOCK,NPAS2 activates circadian gene expressi

[{'xref_id': 'R-HSA-163765', 'xref_name': 'ChREBP activates metabolic gene expression.', 'xref_src_db': 'Reactome'}, {'component_synonym': 'MLX', 'syn_type': 'GENE_SYMBOL'}]
[{'xref_id': 'R-HSA-110056', 'xref_name': 'MAPK3 (ERK1) activation.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-170968', 'xref_name': 'Frs2-mediated activation.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-445144', 'xref_name': 'Signal transduction by L1.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-5210891', 'xref_name': 'Uptake and function of anthrax toxins.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-5673000', 'xref_name': 'RAF activation.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-5674135', 'xref_name': 'MAP2K and MAPK activation.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-5674499', 'xref_name': 'Negative feedback regulation of MAPK pathway.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-5684264', 'xref_name': 'MAP3K8 (TPL2)-dependent MAPK1/3 activation.', 'xref_src_db': 'Reacto

[{'xref_id': 'R-HSA-1257604', 'xref_name': 'PIP3 activates AKT signaling.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-2219530', 'xref_name': 'Constitutive Signaling by Aberrant PI3K in Cancer.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-383280', 'xref_name': 'Nuclear Receptor transcription pathway.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-6811558', 'xref_name': 'PI5P, PP2A and IER3 Regulate PI3K/AKT Signaling.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-8939211', 'xref_name': 'ESR-mediated signaling.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-9009391', 'xref_name': 'Extra-nuclear estrogen signaling.', 'xref_src_db': 'Reactome'}, {'component_synonym': 'ESR2', 'syn_type': 'GENE_SYMBOL'}]
[{'xref_id': 'R-HSA-390696', 'xref_name': 'Adrenoceptors.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-418555', 'xref_name': 'G alpha (s) signalling events.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-5689880', 'xref_name': 'Ub-specific processing proteases.', 'xref_

...

[{'xref_id': 'R-HSA-1299316', 'xref_name': 'TWIK-releated acid-sensitive K+ channel (TASK).', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-5576886', 'xref_name': 'Phase 4 - resting membrane potential.', 'xref_src_db': 'Reactome'}, {'component_synonym': 'KCNK3', 'syn_type': 'GENE_SYMBOL'}]
[{'xref_id': 'R-HSA-163765', 'xref_name': 'ChREBP activates metabolic gene expression.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-196780', 'xref_name': 'Biotin transport and metabolism.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-200425', 'xref_name': 'Carnitine metabolism.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-2426168', 'xref_name': 'Activation of gene expression by SREBF (SREBP).', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-3371599', 'xref_name': 'Defective HLCS causes multiple carboxylase deficiency.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-75105', 'xref_name': 'Fatty acyl-CoA biosynthesis.', 'xref_src_db': 'Reactome'}, {'component_synonym': 'ACACA', 'syn_type': 

[{'component_synonym': 'ERG11', 'syn_type': 'GENE_SYMBOL'}]
[{'xref_id': 'R-HSA-416476', 'xref_name': 'G alpha (q) signalling events.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-420499', 'xref_name': 'Class C/3 (Metabotropic glutamate/pheromone receptors).', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-6794361', 'xref_name': 'Neurexins and neuroligins.', 'xref_src_db': 'Reactome'}, {'component_synonym': 'GRM5', 'syn_type': 'GENE_SYMBOL'}]
[{'xref_id': 'R-HSA-141333', 'xref_name': 'Biogenic amines are oxidatively deaminated to aldehydes by MAOA and MAOB.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-181430', 'xref_name': 'Norepinephrine Neurotransmitter Release Cycle.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-379397', 'xref_name': 'Enzymatic degradation of dopamine by COMT.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-379398', 'xref_name': 'Enzymatic degradation of Dopamine by monoamine oxidase.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-380612', 'xref_name': 

[{'component_synonym': 'ileS', 'syn_type': 'GENE_SYMBOL'}]
[{'xref_id': 'R-HSA-380108', 'xref_name': 'Chemokine receptors bind chemokines.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-418594', 'xref_name': 'G alpha (i) signalling events.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-6798695', 'xref_name': 'Neutrophil degranulation.', 'xref_src_db': 'Reactome'}, {'component_synonym': 'CXCR2', 'syn_type': 'GENE_SYMBOL'}]
[{'xref_id': 'R-HSA-392170', 'xref_name': 'ADP signalling through P2Y purinoceptor 12.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-417957', 'xref_name': 'P2Y receptors.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-418594', 'xref_name': 'G alpha (i) signalling events.', 'xref_src_db': 'Reactome'}, {'component_synonym': 'P2RY12', 'syn_type': 'GENE_SYMBOL'}]
[{'xref_id': 'R-HSA-174178', 'xref_name': 'APC/C:Cdh1 mediated degradation of Cdc20 and other APC/C:Cdh1 targeted proteins in late mitosis/early G1.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-25659

[{'xref_id': 'R-HSA-1257604', 'xref_name': 'PIP3 activates AKT signaling.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-2219530', 'xref_name': 'Constitutive Signaling by Aberrant PI3K in Cancer.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-416550', 'xref_name': 'Sema4D mediated inhibition of cell attachment and migration.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-5673001', 'xref_name': 'RAF/MAP kinase cascade.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-6806942', 'xref_name': 'MET Receptor Activation.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-6807004', 'xref_name': 'Negative regulation of MET activity.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-6811558', 'xref_name': 'PI5P, PP2A and IER3 Regulate PI3K/AKT Signaling.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-8851805', 'xref_name': 'MET activates RAS signaling.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-8851907', 'xref_name': 'MET activates PI3K/AKT signaling.', 'xref_src_db': 'Reactome'}, {'x

[{'xref_id': 'R-HSA-2161517', 'xref_name': 'Abacavir transmembrane transport.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-382556', 'xref_name': 'ABC-family proteins mediated transport.', 'xref_src_db': 'Reactome'}, {'component_synonym': 'ABCB1', 'syn_type': 'GENE_SYMBOL'}]
[{'xref_id': 'R-HSA-1251985', 'xref_name': 'Nuclear signaling by ERBB4.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-3371497', 'xref_name': 'HSP90 chaperone cycle for steroid hormone receptors (SHR).', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-383280', 'xref_name': 'Nuclear Receptor transcription pathway.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-4090294', 'xref_name': 'SUMOylation of intracellular receptors.', 'xref_src_db': 'Reactome'}, {'xref_id': 'R-HSA-9018519', 'xref_name': 'Estrogen-dependent gene expression.', 'xref_src_db': 'Reactome'}, {'component_synonym': 'PGR', 'syn_type': 'GENE_SYMBOL'}]
[{'xref_id': 'R-HSA-1059683', 'xref_name': 'Interleukin-6 signaling.', 'xref_src_db': 'Reactome'

In [51]:
chem2Gene_all

{'CHEMBL5694': [{'xref_id': 'R-HSA-110362',
   'xref_name': 'POLB-Dependent Long Patch Base Excision Repair.',
   'xref_src_db': 'Reactome'},
  {'xref_id': 'R-HSA-162594',
   'xref_name': 'Early Phase of HIV Life Cycle.',
   'xref_src_db': 'Reactome'},
  {'xref_id': 'R-HSA-174414',
   'xref_name': 'Processive synthesis on the C-strand of the telomere.',
   'xref_src_db': 'Reactome'},
  {'xref_id': 'R-HSA-5358565',
   'xref_name': 'Mismatch repair (MMR) directed by MSH2:MSH6 (MutSalpha).',
   'xref_src_db': 'Reactome'},
  {'xref_id': 'R-HSA-5358606',
   'xref_name': 'Mismatch repair (MMR) directed by MSH2:MSH3 (MutSbeta).',
   'xref_src_db': 'Reactome'},
  {'xref_id': 'R-HSA-5651801',
   'xref_name': 'PCNA-Dependent Long Patch Base Excision Repair.',
   'xref_src_db': 'Reactome'},
  {'xref_id': 'R-HSA-5696397',
   'xref_name': 'Gap-filling DNA repair synthesis and ligation in GG-NER.',
   'xref_src_db': 'Reactome'},
  {'xref_id': 'R-HSA-6782210',
   'xref_name': 'Gap-filling DNA repair 

In [67]:
chem2Gene_all['CHEMBL5314'][0]['component_synonym']

'TYRO3'

In [52]:
#filename='data\\normalized_data\\Chem2Gene2Path'
#outfile = open(filename,'wb')

In [53]:
pickle.dump(chem2Gene_all,outfile)
#pickle.dump(named_mechList,outfile)
outfile.close()

In [ ]:
len(ActList_copy['CHEMBL8486'])

In [68]:
def chembl2gene2path(chem2geneList,ActList):    
    for item in chem2geneList:
        #print(item)
        sizeOfitem = len(chem2geneList[item])
        #print(chem2Gene[item])
        #print(chem2geneList[item][sizeOfitem-1]['component_synonym'])
        gene = chem2geneList[item][sizeOfitem-1]['component_synonym']
        for jtem in ActList:
            #print(jtem,'b')
            #print(len(ActList_copy[jtem]))
            for i in range(len(ActList[jtem])):
                if item == ActList.get(jtem)[i]['target_chembl_id']:
                #print(jtem)
                    newkey = {'Protein': gene}
                    ActList[jtem][i].update(newkey)
                #print(ActList_copy.get(jtem)[i]['target_chembl_id'])

    return(ActList)


In [69]:
ActList2gene2Path_3 = chembl2gene2path(chem2Gene_all,ActList)

In [71]:
ActList2gene2Path_3

{'CHEMBL56731': [{'assay_chembl_id': 'CHEMBL763200',
   'assay_type': 'B',
   'bao_label': 'single protein format',
   'pchembl_value': '6.16',
   'target_chembl_id': 'CHEMBL203',
   'target_organism': 'Homo sapiens',
   'type': 'IC50',
   'value': '0.7',
   'Protein': 'EGFR'}],
 'CHEMBL1076347': [{'assay_chembl_id': 'CHEMBL1819920',
   'assay_type': 'B',
   'bao_label': 'single protein format',
   'pchembl_value': '7.89',
   'target_chembl_id': 'CHEMBL2409',
   'target_organism': 'Homo sapiens',
   'type': 'IC50',
   'value': '13.0',
   'Protein': 'EPHX2'}],
 'CHEMBL332898': [{'assay_chembl_id': 'CHEMBL760831',
   'assay_type': 'B',
   'bao_label': 'single protein format',
   'pchembl_value': '6.30',
   'target_chembl_id': 'CHEMBL1827',
   'target_organism': 'Homo sapiens',
   'type': 'IC50',
   'value': '0.5',
   'Protein': 'PDE5A'},
  {'assay_chembl_id': 'CHEMBL4398715',
   'assay_type': 'B',
   'bao_label': 'single protein format',
   'pchembl_value': '6.22',
   'target_chembl_id':

In [73]:
mechList_2 = chembl2gene2path(chem2Gene_all,mechList)

In [74]:
mechList_2

{'CHEMBL2360079': [{'mechanism_of_action': 'Dopamine D2 receptor antagonist',
   'target_chembl_id': 'CHEMBL217',
   'Protein': 'DRD2'}],
 'CHEMBL1275': [{'mechanism_of_action': 'Dopamine receptor agonist',
   'target_chembl_id': 'CHEMBL2096905',
   'Protein': 'DRD2'}],
 'CHEMBL3989821': [{'mechanism_of_action': 'Muscle-type nicotinic acetylcholine receptor antagonist',
   'target_chembl_id': 'CHEMBL2362997',
   'Protein': 'CHRNA1'}],
 'CHEMBL1713': [{'mechanism_of_action': 'Serotonin 2a (5-HT2a) receptor antagonist',
   'target_chembl_id': 'CHEMBL224',
   'Protein': 'HTR2A'},
  {'mechanism_of_action': 'D2-like dopamine receptor antagonist',
   'target_chembl_id': 'CHEMBL2331075',
   'Protein': 'DRD2'}],
 'CHEMBL2062262': [{'mechanism_of_action': 'Adrenergic receptor alpha-1 agonist',
   'target_chembl_id': 'CHEMBL2094251',
   'Protein': 'ADRA1A'}],
 'CHEMBL258805': [{'mechanism_of_action': 'Cyclin-dependent kinase 2 inhibitor',
   'target_chembl_id': 'CHEMBL301',
   'Protein': 'CDK2'}

In [81]:
def chem2gene2path_rel(named_chem2geneList,itmpGraph):
    for item in named_chem2geneList:
        itemLen = len(named_chem2geneList[item])-1
        #print(itemLen)
        #break
        for j in range(itemLen):
            #print(named_chem2geneList)
            itmpGraph.add_association(MicroRna(namespace='HP', name=named_chem2geneList[item][itemLen]['component_synonym']),
                                      Rna(namespace='Pathway',name=named_chem2geneList[item][j]['xref_name']),
                                      citation='ChEMBL database', evidence='ChEMBL query',
                                      Reactome=named_chem2geneList[item][j]['xref_id'])

    return(itmpGraph)

In [82]:
itmp_Graph = pybel.BELGraph(name='itmpGraph')

In [83]:
test = chem2gene2path_rel(chem2Gene_all,itmp_Graph)

In [ ]:
chem2Gene['CHEMBL203'][len(chem2Gene['CHEMBL203'])-1]['component_synonym']

In [ ]:
len(ActList2gene2Path['CHEMBL56731'])

In [ ]:
chem2Gene